# Klasyfikacja bankructwa na podstawie danych ze zbioru "Company Bankruptcy Prediction"

Julia Kaznowska, Piotr Wilczyński <br>
02/04/2022 <br>
Politechnika Warszawska, Wydział Matematyki i Nauk Informacyjnych, Wstęp do uczenia maszynowego

## Import niezbędnych bibliotek oraz zbioru danych

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# zbior danych
df = pd.read_csv("data.csv")

# wyświetlanie wizualizacji
%matplotlib inline

## Wstępne informacje o danych

In [2]:
df.head()

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,1,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,1,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490


In [3]:
df.shape

(6819, 96)

Dane mają 96 kolumn i 6819 rekordów.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6819 entries, 0 to 6818
Data columns (total 96 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   Bankrupt?                                                 6819 non-null   int64  
 1    ROA(C) before interest and depreciation before interest  6819 non-null   float64
 2    ROA(A) before interest and % after tax                   6819 non-null   float64
 3    ROA(B) before interest and depreciation after tax        6819 non-null   float64
 4    Operating Gross Margin                                   6819 non-null   float64
 5    Realized Sales Gross Margin                              6819 non-null   float64
 6    Operating Profit Rate                                    6819 non-null   float64
 7    Pre-tax net Interest Rate                                6819 non-null   float64
 8    After-tax net Int

Nie ma oczywistych braków danych, ale widzimy dwie wyróżniające się wartości: Net Income Flag i Liability-Assets Flag. Obie w przeciwieństwie do wszystkich innych zmiennych opisujących są całkowite a nie zmiennoprzecinkowe. Zobaczmy ich wartości.

In [5]:
unique, counts = np.unique(df[" Liability-Assets Flag"], return_counts=True)
unique, counts

(array([0, 1], dtype=int64), array([6811,    8], dtype=int64))

In [6]:
unique, counts = np.unique(df[" Net Income Flag"], return_counts=True)
unique, counts

(array([1], dtype=int64), array([6819], dtype=int64))

Zmienna "Net Income Flag" zawsze ma wartość 1, zatem nie ma wpływu na predykcyjność. Możemy się jej pozbyć. Zmienna "Liability-Assets Flag" ma wartość 1 tylko w 8 przypadkach, zobaczmy jakich.

In [7]:
df.loc[df[" Liability-Assets Flag"] == 1]

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
56,1,0.066933,0.057185,0.054821,0.601861,0.601861,0.998825,0.796779,0.808717,0.302760,...,0.525651,5.803037e-03,0.623648,0.601857,1.000000,0.182790,0.026763,0.565021,1,0.009178
1869,1,0.392775,0.432239,0.432946,0.586921,0.586921,0.998776,0.797126,0.809068,0.303470,...,0.722761,2.417803e-03,0.622734,0.586923,0.976180,0.000000,0.026703,0.564698,1,0.009879
1870,1,0.277726,0.314708,0.307351,0.596621,0.596650,0.998976,0.797176,0.809113,0.303138,...,0.664814,3.231135e-03,0.623270,0.596619,0.902744,0.199162,0.026755,0.564978,1,0.008950
2001,1,0.438795,0.090166,0.464586,0.540776,0.540776,0.997789,0.790787,0.802967,0.294457,...,0.411809,1.109791e-02,0.625487,0.540775,0.996912,0.209222,0.026779,0.565098,1,0.008753
2470,1,0.404036,0.223615,0.430055,0.586611,0.586611,0.998568,0.796179,0.808154,0.302249,...,0.572881,8.658754e-03,0.623173,0.586607,0.916329,0.218785,0.026745,0.564930,1,0.008500
2735,0,0.436894,0.453718,0.479522,0.585062,0.585062,0.998495,0.796770,0.808785,0.303434,...,0.747290,4.202211e-04,0.557613,0.585059,0.885473,0.133503,0.026744,0.564922,1,0.009546
6613,0,0.279676,0.283362,0.303014,0.637520,0.637520,0.998785,0.797055,0.809000,0.303325,...,0.705559,3.030000e+09,0.623292,0.637516,0.841826,0.265220,0.026791,0.565158,1,0.000000
6640,1,0.196802,0.211023,0.221425,0.598056,0.598056,0.998933,0.796144,0.808149,0.301423,...,0.519388,1.758765e-02,0.623465,0.598051,0.856906,0.259280,0.026769,0.565052,1,0.003946


Flaga Liability-Assets jest ustawiona na 1 zarówno kiedy bakrupt jest 1 jak i 0. Przy eliminacji outlierów i tak zamienimy flagę 1 na 0 (bo jest ich mniej niż 2.5%). Zmienna ta również nie będzie miała wartości predykcyjnej. Można ją zatem usunąć.

In [8]:
df = df.drop(" Liability-Assets Flag", axis = 1)
df = df.drop(" Net Income Flag", axis = 1)

In [9]:
100*df.loc[df["Bankrupt?"] == 1].shape[0]/df.shape[0] 

3.2262795131250916

Zaledwie lekko ponad 3% rekordów jest oznaczone flagą bankurpt.

## Podział zbioru danych na dane budujące i do walidacji

In [10]:
y = np.array(df["Bankrupt?"])
X = df.drop(["Bankrupt?"], axis = 1)
X_build, X_val, y_build, y_val = train_test_split(
    X, y, stratify=y, test_size=0.3, random_state=321
)
X_train, X_test, y_train, y_test = train_test_split(
    X_build, y_build, stratify=y_build, test_size=0.3, random_state=123
)

Eksport danych do walidacji

In [11]:
df_val = X_val.copy()
df_val["Bankrupt?"] = y_val.copy()
df_val.to_csv("data_val.csv")

## Prosty pre-processing danych

In [12]:
df_train = X_train.copy()
df_train["Bankrupt?"] = y_train.copy()
df_test = X_test.copy()
df_test["Bankrupt?"] = y_test.copy()

###  Outliery

Wartości poniżej precentyla 2.5 i powyżej 97.5 będziemy zastępować wartościami skrajnymi.

In [13]:
from sklearn.base import BaseEstimator, TransformerMixin

class outliers(BaseEstimator, TransformerMixin):
    def fit(self, X):
        return self
    def transform(self, X):
        y_temp = X["Bankrupt?"]
        X = X.drop("Bankrupt?", axis = 1)
        for col in X.columns:
            upper_lim = X[col].quantile(.975)
            lower_lim = X[col].quantile(.025)
            X[col] = np.where(X[col] < upper_lim, X[col], upper_lim)
            X[col] = np.where(X[col] > lower_lim, X[col], lower_lim)
        X["Bankrupt?"] = y_temp
        return X

### Zmiana kierunku korelacji

Użyjemy korelacji Spearmana. Wszystkie zmienne przekształcimy tak, aby były dodatnio skorelowane ze zmienną celu.

In [14]:
from scipy.stats import spearmanr

class direction_change(BaseEstimator, TransformerMixin):
    def fit(self, X):
        return self
    def transform(self, X):
        y_temp = X["Bankrupt?"]
        X = X.drop("Bankrupt?", axis = 1)
        for col in X:
            for col in X.columns:
                if (spearmanr(X[col], y_temp)[0] < 0):
                    X[col] = -X[col]
        #X["?Bankrupt"] = y_temp
        X.insert(0, "Bankrupt?",y_temp)
        return X

### Normalizacja min-max i pipeline

Aby znormalizować wartości użyjemy funkcji MinMaxScaler z biblioteki sklearn.

Wprowadzamy wszystkie przekształcenia do pipelina.

In [15]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('outliers', outliers()),
    ('direction_change', direction_change()),
    ('minmax', MinMaxScaler())
])

In [16]:
df_train = pd.DataFrame(pipe.fit_transform(df_train), columns = df.columns)
df_train["Bankrupt?"] = np.int64(df_train["Bankrupt?"])
df_test = pd.DataFrame(pipe.fit_transform(df_test), columns = df.columns)
df_test["Bankrupt?"] = np.int64(df_test["Bankrupt?"])

In [17]:
y_train = np.array(df_train["Bankrupt?"])
X_train = df_train.drop(["Bankrupt?"], axis = 1)

y_test = np.array(df_test["Bankrupt?"])
X_test = df_test.drop(["Bankrupt?"], axis = 1)

## Uczenie klasyfikatorów

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

Wybieramy 3 klasyfikatory i ustawiamy hiperparametry:

In [19]:
model1 = GradientBoostingClassifier(learning_rate=0.2)
model2 = RandomForestClassifier(max_depth = 13)
model3 = DecisionTreeClassifier(max_features = 'sqrt')

In [20]:
model1.fit(X_train, y_train)
model2.fit(X_train, y_train)
model3.fit(X_train, y_train)

DecisionTreeClassifier(max_features='sqrt')

## Ocena jakości klasyfikatorów

Dokonujemy oceny jakości klasyfikatorów. Zdecydowaliśmy się na scoringi `accuracy`, `precision` oraz `recall`.

In [21]:
def print_scores(s1, s2, s3):
    print(f"accuracy: mean = {np.round(np.mean(s1), 4)}, std = {np.round(np.std(s1), 4)}")
    print(f"precision: mean = {np.round(np.mean(s2), 4)}, std = {np.round(np.std(s2), 4)}")
    print(f"recall: mean = {np.round(np.mean(s3), 4)}, std = {np.round(np.std(s3), 4)}")

### Gradient Boosting Classifier

In [22]:
train_acc = cross_val_score(model1, X_train, y_train, scoring='accuracy', cv = 4)
train_pr = cross_val_score(model1, X_train, y_train, scoring='precision', cv = 4)
train_rec = cross_val_score(model1, X_train, y_train, scoring='recall', cv = 4)

In [23]:
print_scores(train_acc, train_pr, train_rec)

accuracy: mean = 0.9662, std = 0.0031
precision: mean = 0.4815, std = 0.0643
recall: mean = 0.2963, std = 0.0786


### Random Forest Classifier

In [24]:
train_acc = cross_val_score(model2, X_train, y_train, scoring='accuracy', cv = 4)
train_pr = cross_val_score(model2, X_train, y_train, scoring='precision', cv = 4)
train_rec = cross_val_score(model2, X_train, y_train, scoring='recall', cv = 4)

In [25]:
print_scores(train_acc, train_pr, train_rec)

accuracy: mean = 0.9722, std = 0.001
precision: mean = 0.7896, std = 0.1723
recall: mean = 0.2593, std = 0.0454


### Decision Tree Classifier

In [26]:
train_acc = cross_val_score(model3, X_train, y_train, scoring='accuracy', cv = 4)
train_pr = cross_val_score(model3, X_train, y_train, scoring='precision', cv = 4)
train_rec = cross_val_score(model3, X_train, y_train, scoring='recall', cv = 4)

In [27]:
print_scores(train_acc, train_pr, train_rec)

accuracy: mean = 0.95, std = 0.0058
precision: mean = 0.3382, std = 0.0428
recall: mean = 0.3056, std = 0.0844


## Wnioski

Wszystkie modele posiadają bardzo wysokie `accuracy` oraz bardzo niski `recall`. Powodem tego jest ekstremalnie niska liczebność przypadków z flagą 1 dla zmiennej targetowanej. Nawet gdyby model przewidywał same wartości 0, to `accuracy` byłoby na poziomie około 0.965. Z powodu małej próbki przedstawiającej firmy, które zbankrutowały, odpowiednie opisanie ich wszystkich jest trudne. <br>
Modele poradziły sobie średnio lepiej w przypadku `precision`. Najlepszy wynik osiągnął `Random Forest Classifier`, który w 70% nie mylił się, klasyfikując firmę jako bankruta. Inne modele osiągnęły znacznie gorsze wyniki. Z tego powodu uznajemy zatem, iż `Random Forest Classifier` jest najlepszy spośród wszystkich modeli z tak ustawionymi hiperparametrami.

## Sprawdzenie jakości dla danych testowych

In [28]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [29]:
def print_scores_test(s1, s2, s3):
    print("accuracy:", np.round(s1, 4))
    print("precision:", np.round(s2, 4))
    print("recall:", np.round(s3, 4))

### Gradient Boosting Classifier

In [30]:
test_acc = accuracy_score(y_test, model1.predict(X_test))
test_pr = precision_score(y_test, model1.predict(X_test))
test_rec = recall_score(y_test, model1.predict(X_test))

In [31]:
print_scores_test(test_acc, test_pr, test_rec)

accuracy: 0.9406
precision: 0.2468
recall: 0.413


### Random Forest Classifier

In [32]:
test_acc = accuracy_score(y_test, model2.predict(X_test))
test_pr = precision_score(y_test, model2.predict(X_test))
test_rec = recall_score(y_test, model2.predict(X_test))

In [33]:
print_scores_test(test_acc, test_pr, test_rec)

accuracy: 0.9665
precision: 0.4583
recall: 0.2391


### Decision Tree Classifier

In [34]:
test_acc = accuracy_score(y_test, model3.predict(X_test))
test_pr = precision_score(y_test, model3.predict(X_test))
test_rec = recall_score(y_test, model3.predict(X_test))

In [35]:
print_scores_test(test_acc, test_pr, test_rec)

accuracy: 0.8848
precision: 0.0719
recall: 0.2174


## Wnioski

Dla danych testowych `accuracy` jest dalej bardzo wysokie, a `recall` dość niski. Można zauważyć, że `Gradient Boosting Classifier` i `Decision Tree Classifier` osiągają podobne wyniki w porównaniu z danymi treningowymi. `Random Forest Classifier` natomiast ma niższy `precision` i zdecydowanie niższy `recall`. Najprawdopodobniej model ten został przetrenowany. 

# Walidacja

Dominik Kędzierski, Krzysztof Wodnicki

Praca wykonana przez zespół budujący jest ogólnie bardzo dobra. Niestety na kilka niedociągnięć na leży zwrócić uwagę.

## Preprocessing

Wykorzystanie pipelineów, funkcji `fit` i `transform` nie jest do końca takie jak zakładali to twórcy API. Żeby mozna było wygodnie korzystać z przygotowanego pipelineu funkcja `fit` powinna wyliczać wszystkie parametry na podstawie danych treningowych, tak aby na przykład wartości maksymalna i minimalna potrzebne do określenia outlierów, czy kolumny których kierunek należy zmienić zależały tylko od danych treningówych.

In [36]:
class val_outliers(BaseEstimator, TransformerMixin):
    def fit(self, X):
        self.upper_lim={}
        self.lower_lim={}
        for col in X.columns:
            self.upper_lim[col] = X[col].quantile(.975)
            self.lower_lim[col] = X[col].quantile(.025)
        return self
    def transform(self, X):
        y_temp = X["Bankrupt?"]
        X = X.drop("Bankrupt?", axis = 1)
        for col in X.columns:
            X[col] = np.where(X[col] < self.upper_lim[col], X[col], self.upper_lim[col])
            X[col] = np.where(X[col] > self.lower_lim[col], X[col], self.lower_lim[col])
        X["Bankrupt?"] = y_temp
        return X

In [37]:
class val_direction_change(BaseEstimator, TransformerMixin):
    def fit(self, X):
        self.dir_change=set()
        y_temp = X["Bankrupt?"]
        X = X.drop("Bankrupt?", axis = 1)
        for col in X:
            for col in X.columns:
                if (spearmanr(X[col], y_temp)[0] < 0):
                    self.dir_change.add(col)
        #X["?Bankrupt"] = y_temp
        X.insert(0, "Bankrupt?",y_temp)
        return self
    def transform(self, X):
        y_temp = X["Bankrupt?"]
        X = X.drop("Bankrupt?", axis = 1)
        for col in X:
            for col in X.columns:
                if (col in self.dir_change):
                    X[col] = -X[col]
        #X["?Bankrupt"] = y_temp
        X.insert(0, "Bankrupt?",y_temp)
        return X

In [38]:
val_pipe = Pipeline([
    ('outliers', val_outliers()),
    ('direction_change', val_direction_change()),
    ('minmax', MinMaxScaler())
])

Warto zwrócić też uwagę, że zespół budujący przez pomyłkę przy preprocessingu wywołał funkcję `fit_transform` pipelineu na zbiorze testowym. Przez to pipeline dopasował się do zbioru testowego, co mogło później zaburzyć wyniki.

In [39]:
val_pipe.fit(df_train)

Pipeline(steps=[('outliers', val_outliers()),
                ('direction_change', val_direction_change()),
                ('minmax', MinMaxScaler())])

In [40]:
df_train = pd.DataFrame(val_pipe.transform(df_train), columns = df.columns)
df_train["Bankrupt?"] = np.int64(df_train["Bankrupt?"])

df_test = pd.DataFrame(val_pipe.transform(df_test), columns = df.columns)
df_test["Bankrupt?"] = np.int64(df_test["Bankrupt?"])

df_val = pd.DataFrame(val_pipe.transform(df_val), columns = df.columns)
df_val["Bankrupt?"] = np.int64(df_val["Bankrupt?"])

In [41]:
y_train = np.array(df_train["Bankrupt?"])
X_train = df_train.drop(["Bankrupt?"], axis = 1)

y_test = np.array(df_test["Bankrupt?"])
X_test = df_test.drop(["Bankrupt?"], axis = 1)

y_val = np.array(df_val["Bankrupt?"])
X_val = df_val.drop(["Bankrupt?"], axis = 1)

Dla danych po poprawnie użytym preprocessingu warto jescze raz policzyć wskaźniki jakości klasyfikatora.

### Gradient Boosting Classifier

In [42]:
test_acc = accuracy_score(y_test, model1.predict(X_test))
test_pr = precision_score(y_test, model1.predict(X_test))
test_rec = recall_score(y_test, model1.predict(X_test))

In [43]:
print_scores_test(test_acc, test_pr, test_rec)

accuracy: 0.9351
precision: 0.2299
recall: 0.4348


### Random Forest Classifier

In [44]:
test_acc = accuracy_score(y_test, model2.predict(X_test))
test_pr = precision_score(y_test, model2.predict(X_test))
test_rec = recall_score(y_test, model2.predict(X_test))

In [45]:
print_scores_test(test_acc, test_pr, test_rec)

accuracy: 0.9665
precision: 0.4583
recall: 0.2391


### Decision Tree Classifier

In [46]:
test_acc = accuracy_score(y_test, model3.predict(X_test))
test_pr = precision_score(y_test, model3.predict(X_test))
test_rec = recall_score(y_test, model3.predict(X_test))

In [47]:
print_scores_test(test_acc, test_pr, test_rec)

accuracy: 0.8841
precision: 0.0714
recall: 0.2174


Jak widać, wszystkie modele radzą sobie podobnie jak przy oryginalnej pracy budujących.

## Sprawdzenie jakości dla danych walidujących

### Gradient Boosting Classifier

In [48]:
val_acc = accuracy_score(y_val, model1.predict(X_val))
val_pr = precision_score(y_val, model1.predict(X_val))
val_rec = recall_score(y_val, model1.predict(X_val))

In [49]:
print_scores_test(val_acc, val_pr, val_rec)

accuracy: 0.9663
precision: 0.0
recall: 0.0


### Random Forest Classifier

In [50]:
val_acc = accuracy_score(y_val, model2.predict(X_val))
val_pr = precision_score(y_val, model2.predict(X_val))
val_rec = recall_score(y_val, model2.predict(X_val))

In [51]:
print_scores_test(val_acc, val_pr, val_rec)

accuracy: 0.9677
precision: 0.0
recall: 0.0


### Decision Tree Classifier

In [52]:
val_acc = accuracy_score(y_val, model3.predict(X_val))
val_pr = precision_score(y_val, model3.predict(X_val))
val_rec = recall_score(y_val, model3.predict(X_val))

In [53]:
print_scores_test(val_acc, val_pr, val_rec)

accuracy: 0.0332
precision: 0.0323
recall: 1.0


Na danych walidujących modele osiągają podobne wyniki `accuracy` jak na danych testowych. Natomiast `precision` i `recall` znacząco różnią się dla danych walidujących i testowych.

In [54]:
print(sum(model1.predict(X_val)))
print(sum(model2.predict(X_val)))
print(sum(model3.predict(X_val)))

3
0
2044


Jak widać, dla danych walidacyjnych, modele prawie nigdy nie klasyfikują rekordu jako bankruta. Jest to ciekawe zachowanie, które prawdopodobnie wynika z małej ilości rekorów z flagą `bankrupt`.

## Podsumowanie

Preprocessing przeprowadzony przez zespół budujący wymagał drobnych poprawek. Oprócz tego praca wykonana została bardzo porządnie. Niestety ze względu na mały udział rekorów z flagą `bankrupt` w zbiorze danych, ciężko przygotować wysokiej jakości model.

# Odpowiedź na walidację
Julia Kaznowska, Piotr Wilczyński

Zgadzamy się z walidatorami, że ich poprawki w pipelinie są lepszym rozwiązaniem od oryginalnego. Będziemy ich zatem używać do trenowania modelu. Dodatkowo zmienimy klasyfikatory na takie, ktore opierają się o boosting, ponieważ takie (przynajmniej w teorii) powinny lepiej sobie radzić z małymi zbiorami danych (rekordy z flagą bankurpt, będą mogły być wybierane parokrotnie).

In [55]:
corrected_pipe = Pipeline([
    ('outliers', val_outliers()),
    ('direction_change', val_direction_change()),
    ('minmax', MinMaxScaler())
])

In [56]:
corrected_pipe.fit(df_train)

Pipeline(steps=[('outliers', val_outliers()),
                ('direction_change', val_direction_change()),
                ('minmax', MinMaxScaler())])

In [57]:
df_train = pd.DataFrame(corrected_pipe.transform(df_train), columns = df.columns)
df_train["Bankrupt?"] = np.int64(df_train["Bankrupt?"])

df_test = pd.DataFrame(corrected_pipe.transform(df_test), columns = df.columns)
df_test["Bankrupt?"] = np.int64(df_test["Bankrupt?"])

In [58]:
y_train = np.array(df_train["Bankrupt?"])
X_train = df_train.drop(["Bankrupt?"], axis = 1)

y_test = np.array(df_test["Bankrupt?"])
X_test = df_test.drop(["Bankrupt?"], axis = 1)

In [59]:
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
model1 = GradientBoostingClassifier(learning_rate=0.2)
model4 = XGBClassifier(max_depth = 13, use_label_encoder =False, verbosity = 0)
model5 = AdaBoostClassifier(n_estimators = 100)

In [60]:
model1.fit(X_train, y_train)
model4.fit(X_train, y_train)
model5.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=100)

### Gradient Boosting Classifier

In [61]:
train_acc = cross_val_score(model1, X_train, y_train, scoring='accuracy', cv = 4)
train_pr = cross_val_score(model1, X_train, y_train, scoring='precision', cv = 4)
train_rec = cross_val_score(model1, X_train, y_train, scoring='recall', cv = 4)

test_acc = accuracy_score(y_test, model1.predict(X_test))
test_pr = precision_score(y_test, model1.predict(X_test))
test_rec = recall_score(y_test, model1.predict(X_test))

In [62]:
print_scores(train_acc, train_pr, train_rec)

accuracy: mean = 0.9671, std = 0.0032
precision: mean = 0.4995, std = 0.0676
recall: mean = 0.287, std = 0.0844


In [63]:
print_scores_test(test_acc, test_pr, test_rec)

accuracy: 0.942
precision: 0.2466
recall: 0.3913


### XGBoost

In [64]:
train_acc = cross_val_score(model4, X_train, y_train, scoring='accuracy', cv = 4)
train_pr = cross_val_score(model4, X_train, y_train, scoring='precision', cv = 4)
train_rec = cross_val_score(model4, X_train, y_train, scoring='recall', cv = 4)

test_acc = accuracy_score(y_test, model4.predict(X_test))
test_pr = precision_score(y_test, model4.predict(X_test))
test_rec = recall_score(y_test, model4.predict(X_test))

In [65]:
print_scores(train_acc, train_pr, train_rec)

accuracy: mean = 0.9716, std = 0.0016
precision: mean = 0.6264, std = 0.059
recall: mean = 0.3148, std = 0.0556


In [66]:
print_scores_test(test_acc, test_pr, test_rec)

accuracy: 0.9588
precision: 0.3333
recall: 0.2826


### AdaBoost

In [67]:
train_acc = cross_val_score(model5, X_train, y_train, scoring='accuracy', cv = 4)
train_pr = cross_val_score(model5, X_train, y_train, scoring='precision', cv = 4)
train_rec = cross_val_score(model5, X_train, y_train, scoring='recall', cv = 4)

test_acc = accuracy_score(y_test, model5.predict(X_test))
test_pr = precision_score(y_test, model5.predict(X_test))
test_rec = recall_score(y_test, model5.predict(X_test))

In [68]:
print_scores(train_acc, train_pr, train_rec)

accuracy: mean = 0.9671, std = 0.0026
precision: mean = 0.4906, std = 0.0582
recall: mean = 0.3426, std = 0.0404


In [69]:
print_scores_test(test_acc, test_pr, test_rec)

accuracy: 0.9644
precision: 0.3529
recall: 0.1304


## Podsumowanie

Mimo poprawek walidacji, `Gradient Boost Classifier` dla danych testowych niestety nie osiąga dużo lepszych wyników. `XGBoost` oraz `AdaBoost` dla danych treningowych radzą sobie lepiej niż poprzednio użyty `Decision Tree Classifier`. Mają one również lepszy wskaźnik `recall` niż dla `Random Forest Classifier`, przypłacając to natomiast gorszym `precision`. Dla danych testowych scoringi spadają, nie dają nam poszukiwej wydajności.